In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import math

In [59]:
def causal_mask(size):
    masks = torch.triu(torch.ones((1, size, size)), diagonal=1).int()

    return masks == 0

In [60]:
# Load Dataset
from torch.utils.data import Dataset


class BilingualDataset(Dataset):
    def __init__(self, ds, src_tokenizer, tgt_tokenizer, src_lang, tgt_lang, seq_len):
        super().__init__()

        self.seq_len = seq_len

        self.ds = ds
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

        self.sos_token = torch.tensor(
            [self.tgt_tokenizer.token_to_id("[SOS]")], dtype=torch.int64
        )
        self.eos_token = torch.tensor(
            [self.tgt_tokenizer.token_to_id("[EOS]")], dtype=torch.int64
        )
        self.pad_token = torch.tensor(
            [self.tgt_tokenizer.token_to_id("[PAD]")], dtype=torch.int64
        )

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        src_target_pair = self.ds[idx]

        src_text = src_target_pair["translation"][self.src_lang]
        tgt_text = src_target_pair["translation"][self.tgt_lang]

        # Tokenize
        src_tokens = self.src_tokenizer.encode(src_text).ids
        tgt_tokens = self.tgt_tokenizer.encode(tgt_text).ids

        # Calculate the number of padding tokens
        src_pad_len = self.seq_len - len(src_tokens) - 2  # 2 for sos and eos
        tgt_pad_len = self.seq_len - len(tgt_tokens) - 1  # 1 for sos

        # Check padding is not negative, if it is, truncate the sequence
        if src_pad_len < 0:
            src_tokens = src_tokens[: self.seq_len - 2]
            src_pad_len = 0
        if tgt_pad_len < 0:
            tgt_tokens = tgt_tokens[: self.seq_len - 1]
            tgt_pad_len = 0

        # Add SOS, EOS, PAD tokens
        encoder_tokens = torch.cat(
            (
                self.sos_token,
                torch.tensor(src_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * src_pad_len, dtype=torch.int64),
            ),
            dim=0,
        )

        decoder_tokens = torch.cat(
            (
                self.sos_token,
                torch.tensor(tgt_tokens, dtype=torch.int64),
                torch.tensor([self.pad_token] * tgt_pad_len, dtype=torch.int64),
            ),
            dim=0,
        )

        label = torch.cat(
            (
                torch.tensor(tgt_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * tgt_pad_len, dtype=torch.int64),
            ),
            dim=0,
        )

        # Check the size of the tensors, make sure they are the same size
        assert src_tokens.size(0) == self.seq_len
        assert tgt_tokens.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        return {
            "encoder_tokens": encoder_tokens,  # (seq_len)
            "decoder_tokens": decoder_tokens,  # (seq_len)
            "encoder_mask": (encoder_tokens != self.pad_token)
            .unsqueeze(0)
            .unsqueeze(0)
            .int(),  # (1, 1, seq_len)
            "decoder_mask": (decoder_tokens != self.pad_token).unsqueeze(0).int()
            & causal_mask(self.seq_len),  # (1, seq_len, seq_len)
            "label": label,
        }

In [61]:
causal_mask(10)

tensor([[[ True, False, False, False, False, False, False, False, False, False],
         [ True,  True, False, False, False, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False, False, False],
         [ True,  True,  True,  True, False, False, False, False, False, False],
         [ True,  True,  True,  True,  True, False, False, False, False, False],
         [ True,  True,  True,  True,  True,  True, False, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True,  True, False, False],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True, False],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True]]])

# Transformer

Let's build the transformer model from bottom up. We will start with the basic building blocks and then put them together to build the transformer model.


In [62]:
# Layer Normalization
class LayerNormalization(nn.Module):
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features))
        self.bias = nn.Parameter(torch.zeros(features))

    def forward(self, x):
        # X: (batch_size, seq_len, features)
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)

        return self.alpha * (x - mean) / (std + self.eps) + self.bias

In [63]:
# Embedding Layer
class InputEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.embedding_size = embedding_size

    def forward(self, x):
        # x: (batch_size, seq_len)
        # Scaling the embedding by sqrt(embedding_size) as per the paper
        return self.embedding(x) * math.sqrt(self.embedding_size)

In [64]:
# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()

        # pe: (max_len, d_model)
        pe = torch.zeros(max_len, d_model)

        # position: (max_len, 1)
        position = torch.arange(0, max_len).unsqueeze(1).float()

        # div_term: (d_model // 2)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)
        )

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # pe: (1, max_len, d_model)
        pe = pe.unsqueeze(0)

        self.register_buffer("pe", pe)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        x = x + self.pe[:, : x.size(1)]
        return self.dropout(x)

In [65]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()

        self.d_model = d_model
        self.n_heads = n_heads

        assert d_model % n_heads == 0, "d_model should be divisible by n_heads"
        self.head_dim = d_model // n_heads

        self.qkv_ln = nn.Linear(d_model, 3 * d_model)
        self.out = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):

        q, k, v = map(
            lambda t: t.view(
                t.size(0), t.size(1), self.n_heads, self.head_dim
            ).transpose(1, 2),
            self.qkv_ln(x).chunk(3, dim=-1),
        )

        # Calculate the attention scores
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)

        # Apply mask
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        # Apply softmax
        scores = F.softmax(scores, dim=-1)

        # Apply dropout
        scores = self.dropout(scores)

        # Get attention
        attention = torch.matmul(scores, v)

        # Concat
        attention = (
            attention.transpose(1, 2)
            .contiguous()
            .view(x.size(0), x.size(1), self.d_model)
        )

        return self.out(attention)

In [66]:
# Residual Layer and Layer Normalization
class ResidualLayer(nn.Module):
    def __init__(self, features, dropout=0.1):
        super().__init__()

        self.norm = LayerNormalization(features)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return self.dropout(self.norm(x + sublayer(x)))  # Post Layer Normalization

In [67]:
# Feed Forward Layer
class FFN(nn.Module):
    def __init__(
        self,
        d_model,
        d_ff,
        dropout=0.1,
    ):
        super().__init__()

        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

In [68]:
# Encoder Block
class EncoderBlock(nn.Module):
    def __init__(
        self,
        features,
        n_heads,
        d_ff,
        dropout=0.1,
    ):
        super().__init__()

        self.attention = MultiHeadAttention(features, n_heads, dropout)
        self.ffn = FFN(features, d_ff, dropout)
        self.residual = nn.ModuleList(
            [ResidualLayer(features, dropout) for _ in range(2)]
        )

    def forward(self, x, src_mask):
        x = self.residual[0](x, lambda x: self.attention(x, src_mask))
        x = self.residual[1](x, self.ffn)

        return x

In [69]:
# Encoder
class Encoder(nn.Module):
    def __init__(
        self,
        features,
        n_heads,
        d_ff,
        n_layers=8,
        dropout=0.1,
    ):
        super().__init__()

        self.layers = nn.ModuleList(
            [EncoderBlock(features, n_heads, d_ff, dropout) for _ in range(n_layers)]
        )

        self.norm = LayerNormalization(features)

    def forward(self, x, src_mask):
        for layer in self.layers:
            x = layer(x, src_mask)

        return self.norm(x)

In [70]:
# Decoder Block
class DecoderBlock(nn.Module):
    def __init__(
        self,
        features,
        n_heads,
        d_ff,
        dropout=0.1,
    ):
        super().__init__()

        self.attention = MultiHeadAttention(features, n_heads, dropout)
        self.ffn = FFN(features, d_ff, dropout)
        self.residual = nn.ModuleList(
            [ResidualLayer(features, dropout) for _ in range(3)]
        )

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual[0](x, lambda x: self.attention(x, tgt_mask))
        x = self.residual[1](x, lambda x: self.attention(x, src_mask))

        x = self.residual[2](x, self.ffn)

        return x

In [71]:
class Decoder(nn.Module):
    def __init__(self, features, n_heads, d_ff, n_layers=8, dropout=0.1):
        super().__init__()

        self.layers = nn.ModuleList(
            [DecoderBlock(features, n_heads, d_ff, dropout) for _ in range(n_layers)]
        )

        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)

        return self.norm(x)

In [72]:
class Projection(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()

        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        return self.fc(x)

In [81]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        tgt_vocab_size,
        src_embedding_size,
        tgt_embedding_size,
        d_model,
        n_heads,
        d_ff,
        n_layers,
        dropout=0.1,
    ):
        super().__init__()

        self.src_embedding = InputEmbedding(src_vocab_size, src_embedding_size)
        self.tgt_embedding = InputEmbedding(tgt_vocab_size, tgt_embedding_size)

        self.encoder = Encoder(d_model, n_heads, d_ff, n_layers, dropout)
        self.decoder = Decoder(d_model, n_heads, d_ff, n_layers, dropout)

        self.projection = Projection(tgt_vocab_size, d_model)

        # Weight tie
        self.projection.fc.weight = self.tgt_embedding.embedding.weight

    def encode(self, encoder_input, src_mask):
        return self.encoder(self.src_embedding(encoder_input), src_mask)

    def decode(self, decoder_input, encoder_output, src_mask, tgt_mask):
        return self.decoder(
            self.tgt_embedding(decoder_input), encoder_output, src_mask, tgt_mask
        )

    def forward(self, encoder_input, decoder_input, src_mask, tgt_mask):
        encoder_output = self.encode(encoder_input, src_mask)
        decoder_output = self.decode(decoder_input, encoder_output, src_mask, tgt_mask)

        return self.projection(decoder_output)

In [82]:
def build_transformer(
    src_vocab_size,
    tgt_vocab_size,
    src_embedding_size,
    tgt_embedding_size,
    d_model,
    n_heads,
    d_ff,
    n_layers,
    dropout=0.1,
):
    transformer = Transformer(
        src_vocab_size,
        tgt_vocab_size,
        src_embedding_size,
        tgt_embedding_size,
        d_model,
        n_heads,
        d_ff,
        n_layers,
        dropout,
    )

    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return transformer

In [83]:
class Config:
    src_vocab_size = 10000
    tgt_vocab_size = 10000
    src_embedding_size = 512
    tgt_embedding_size = 512
    d_model = 512
    n_heads = 8
    d_ff = 2048
    n_layers = 6
    dropout = 0.1
    seq_len = 128

In [85]:
# test
config = Config()
transformer = build_transformer(
    config.src_vocab_size,
    config.tgt_vocab_size,
    config.src_embedding_size,
    config.tgt_embedding_size,
    config.d_model,
    config.n_heads,
    config.d_ff,
    config.n_layers,
    config.dropout,
)

x = torch.randint(0, 10000, (32, 128))
y = torch.randint(0, 10000, (32, 128))


src_mask = (x != 0).unsqueeze(1).unsqueeze(1)
tgt_mask = causal_mask(128)

assert transformer.encode(x, src_mask).shape == (32, 128, config.d_model)
assert transformer.decode(
    y, transformer.encode(x, src_mask), src_mask, tgt_mask
).shape == (32, 128, config.d_model)
assert transformer(x, y, src_mask, tgt_mask).shape == (32, 128, config.tgt_vocab_size)

In [78]:
# Training the Model

In [86]:
# Load dataset
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace


from torch.utils.data import DataLoader, random_split

ModuleNotFoundError: No module named 'datasets'

In [ ]:
def build_tokenizer(data, vocab_size):
    tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()
    trainer = WordLevelTrainer(
        special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2
    )
    tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
    tokenizer.save(str(tokenizer_path))

    return tokenizer

In [88]:
def get_ds(config):
    ds_raw = load_dataset("wmt16", "de-en", split="train")

    src_tokenizer = build_tokenizer(ds, config.src_vocab_size)
    tgt_tokenizer = build_tokenizer(ds, config.tgt_vocab_size)

    train_ds_size = int(0.9 * len(ds_raw))
    val_ds_size = len(ds_raw) - train_ds_size
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

    train_ds = BilingualDataset(
        train_ds_raw,
        src_tokenizer,
        tgt_tokenizer,
        config["lang_src"],
        config["lang_tgt"],
        config["seq_len"],
    )
    val_ds = BilingualDataset(
        val_ds_raw,
        src_tokenizer,
        tgt_tokenizer,
        config["lang_src"],
        config["lang_tgt"],
        config["seq_len"],
    )

    # Find the maximum sequence length
    max_len_src = 0
    max_len_tgt = 0

    for item in ds_raw:
        src_ids = src_tokenizer.encode(item["translation"][config["lang_src"]]).ids
        tgt_ids = tgt_tokenizer.encode(item["translation"][config["lang_tgt"]]).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))

    print(f"Max length of source sentence: {max_len_src}")
    print(f"Max length of target sentence: {max_len_tgt}")

    train_data_loader = DataLoader(
        train_ds,
        batch_size=config["batch_size"],
        shuffle=True,
        num_workers=config["num_workers"],
    )
    val_data_loader = DataLoader(
        val_ds,
        batch_size=config["batch_size"],
        shuffle=False,
        num_workers=config["num_workers"],
    )

    return {
        "train_dataloader": train_data_loader,
        "val_dataloader": val_data_loader,
        "src_tokenizer": src_tokenizer,
        "tgt_tokenizer": tgt_tokenizer,
    }

In [ ]:
def greedy_decode(model, encoder_input, src_mask, tgt_tokenizer, max_len=128):
    model.eval()

    with torch.no_grad():
        encoder_output = model.encode(encoder_input, src_mask)

        decoder_input = torch.tensor(
            [[tgt_tokenizer.token_to_id("[SOS]")]], dtype=torch.int64
        )
        decoder_mask = torch.ones(1, 1, 1).int()

        for _ in range(max_len):
            decoder_output = model.decode(
                decoder_input, encoder_output, src_mask, decoder_mask
            )

            # Get the last token
            next_token = decoder_output.argmax(dim=-1)[:, -1].unsqueeze(1)

            # Concatenate the next token
            decoder_input = torch.cat((decoder_input, next_token), dim=1)

            # Stop if the next token is EOS
            if next_token.item() == tgt_tokenizer.token_to_id("[EOS]"):
                break

        return decoder_input[:, 1:]

In [ ]:
def validation(
    model, val_dataloader, src_tokenizer, tgt_tokenizer, device, max_len=128
):
    model.eval()

    total_loss = 0
    total_correct = 0
    total = 0

    for batch in val_dataloader:
        encoder_input = batch["encoder_tokens"].to(device)
        decoder_input = batch["decoder_tokens"].to(device)
        label = batch["label"].to(device)
        src_mask = batch["encoder_mask"].to(device)
        tgt_mask = batch["decoder_mask"].to(device)

        output = model(encoder_input, decoder_input, src_mask, tgt_mask)

        loss = F.cross_entropy(
            output.view(-1, output.size(-1)), label.view(-1), ignore_index=0
        )

        total_loss += loss.item()

        # Calculate accuracy
        total_correct += (output.argmax(dim=-1) == label).sum().item()
        total += label.size(0) * label.size(1)

    return total_loss / len(val_dataloader), total_correct / total

In [ ]:
# Training the model !!
def train(config):
    train_dataloader, val_dataloader, src_tokenizer, tgt_tokenizer = get_ds(config)
    model = build_transformer(
        config.src_vocab_size,
        config.tgt_vocab_size,
        config.src_embedding_size,
        config.tgt_embedding_size,
        config.d_model,
        config.n_heads,
        config.d_ff,
        config.n_layers,
        config.dropout,
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, eps=1e-9)
    loss_fn = nn.CrossEntropyLoss(ignore_index=tgt_tokenizer.token_to_id("[PAD]"))

    for epoch in range(config.epochs):
        torch.cuda.empty_cache()  # Clear the cache
        model.train()

        for batch in train_dataloader:
            encoder_input = batch["encoder_tokens"].to(device)
            decoder_input = batch["decoder_tokens"].to(device)
            src_mask = batch["encoder_mask"].to(device)
            tgt_mask = batch["decoder_mask"].to(device)
            label = batch["label"].to(device)

            optimizer.zero_grad()
            output = model(encoder_input, decoder_input, src_mask, tgt_mask)

            loss = loss_fn(output.view(-1, config.tgt_vocab_size), label.view(-1))
            loss.backward()
            optimizer.step()

In [ ]:
def main():
    config = Config()
    train(config)

In [ ]:
# Import warnings
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    main()